품사 판별 (Part of speech tagging)은 주어진 문장에 대하여 단어를 인식하고 각 단어의 품사를 판별하는 과정입니다. KoNLPy는 여러 종류의 품사 판별기를 파이썬 환경에서 이용할 수 있도록 도와줍니다. 

품사 판별기는 주로 말뭉치라 불리는 학습데이터를 기반으로 문장/어절의 패턴을 학습합니다. 

    [('아이오아이', '명사'), ('가', '조사'), ('5', '숫자'), ('년', '명사'), ...]
    
위와 같이 단어 별로 품사가 적혀있는 데이터를 학습에 이용합니다. 이와 같이 학습용 데이터를 이용하는 방법을 supervised learning이라 합니다. 하지만, supervised learning 기반 품사 판별기가 가지는 위험 중 하나는 모르는 단어가 등장했을 때 이를 처리하는 방법입니다. '아이오아이' 분명 단어임에도 학습 때 본 적이 없다면 단어로 인식되지 않을 수 있습니다. 

또 다른 위험 중 하나는 도메인의 특별한 어구들을 알 수 없다는 겁니다. '끝까지간다'는 배우 이선균이 출연한 영화 제목 입니다. 데이터 분석의 입장에서는 '끝까지간다'를 문장이 아닌 단어로 보는 것이 더 적합합니다. 만약 영화리뷰를 분석하고 있다면 '끝까지간다'라는 단어가 여러 번 등장할 것이고, 우리는 리뷰들에 있는 이 단어에 여러번 노출되어 단어로 인식할 것입니다. 하지만 품사 판별기의 목적은 주어진 문장/어절을 알고 있는 단어들로 분해하여 인식하는 것이기 때문에 이를 하나의 단어로 인식하지 않습니다. 목적이 다르죠. 

In [36]:
from konlpy.tag import Twitter

twitter = Twitter()
twitter.pos('아이오아이가 5년뒤에 다시 모였으면 좋겠습니다. 소나기 곡 너무 찡해요')

[('아이오', 'Noun'),
 ('아이', 'Noun'),
 ('가', 'Josa'),
 ('5', 'Number'),
 ('년뒤', 'Noun'),
 ('에', 'Josa'),
 ('다시', 'Noun'),
 ('모였', 'Verb'),
 ('으면', 'Eomi'),
 ('좋', 'Adjective'),
 ('겠', 'PreEomi'),
 ('습니다', 'Eomi'),
 ('.', 'Punctuation'),
 ('소나기', 'Noun'),
 ('곡', 'Noun'),
 ('너무', 'Noun'),
 ('찡', 'Adverb'),
 ('해', 'Noun'),
 ('요', 'Josa')]

In [54]:
twitter.pos('끝까지간다')

[('끝', 'Noun'), ('까지', 'Josa'), ('간다', 'Noun')]

그래서 주어진 문서 집합에서 패턴을 찾아내어 단어를 추출해보려 합니다. 다른 학습데이터는 이용하지 않으며, 통계에 기반하여 단어를 찾아내는 unsupervised learning 방법입니다. 통계 기반으로 단어를 추출하는 방법은 여러가지가 있습니다. 그 중 3가지 방법을 구현해 두었습니다. 

    Accessor Variety
    Branching Entropy
    Cohesion Score


## Branching Entropy

Branching entropy는 Jin and Tanaka-Ishii (2006)[1]이 제안한 방법으로, 중국어의 unsupervised word segmentation 문제에 쓰이던 방법입니다. 중국어는 본래 띄어쓰기 없이 문장을 쓰며, 일반적으로 이용하는 글자의 수가 7,000 ~ 20,000자이기 때문에 문장에서 단어만 분리가 되어도 Part of speech tagging까지도 거의 풀린 셈이 됩니다. 그렇기 때문에 중국어의 자연어처리에서는 word segmentation이라는 분야가 연구되었으며, 중국에서도 새로운 단어는 계속 생성되기 때문에 이를 해결하기 위한 방법으로 unsupervised word segmentation 분야가 연구되었죠. 

Branching entropy의 컨셉은 아래의 그림과 같습니다. 'n'다음에 올 글자를 말해보라고 하면 많은 사람들은 잘 모르겠다고 말할 겁니다. 왜냐면 마음 속에 무슨 단어를 떠올리면서 n을 이야기했는지 모르기 때문이죠. 문맥이 부족합니다. 그래서 n 다음에 올 수 있는 글자의 종류는 많고, 불확실성이 높습니다. 하지만 'nat'다음에는요? 만약 'natur'까지 주어졌다면 다음 글자는 'natur - e' 아니면 'natur - al'의 e 아니면 a가 아닐까요? 문맥이 명확히 주어질수록 다음 글자의 불확실성은 내려갑니다. 

이러한 불확실성을 수치로 나타내는 개념으로 entropy가 있습니다. 불확실성이 낮아질수록 entropy값은 작아집니다. 그런데 'natural'다음에 오는 글자는 다시 여러 종류가 있을 겁니다. 단어의 경계이기 때문에 다른 단어들이 등장할 수 있기 때문이죠. 이처럼 단어의 경계에서 entropy가 높을 것이라 가정하는 방법이 Branching entropy입니다. 

![branching_entropy](./figs/branching_entropy.JPG)

이 방법은 단어의 좌/우에 대하여 모두 적용할 수 있습니다. 이런 재료들은 이후에 Tokenizer의 재료가 됩니다. 토크나이저의 목적은 주어진 문장/어절에서 단어를 분리하는 것이기 때문입니다. 

![bothside_branching](./figs/bothside_branching_entropy.JPG)

## Accessor Variety

Accessor Variety[2]는 Branching Entropy보다 더 일찍 나온 개념입니다. 간단하게 말하면 Accessor Variety는 branching entropy의 discrete 버전입니다. nature, natural 두 단어가 존재한다고 하면 natur + [-e, -a]이기 때문에 right-side의 accessor variety는 len({-e, -a}) = 2 입니다. Branching entropy는 -e와 -a가 몇 번 나왔는지의 빈도수를 센 다음, 이를 이용하여 entropy를 계산하는게 차이입니다. 

Accessor Variety는 discrete value이기 때문에 Conditional Random Field의 potential function을 구성하는데 편리합니다. 그래서 Conditional Random Field를 이용하는 Chinese word segmentation에서 features를 생성하는데 자주 이용되었습니다. 

## Cohesion Score

Branching Entropy와 Accessor Variety는 단어의 좌/우의 경계에 등장하는 글자들의 정보를 이용하여 단어의 경계를 판단하는 exterior boundary scoring 방법입니다. 이것과 다르게 단어를 구성하는 글자들이 얼마나 함께 자주 등장하는지의 정보로도 단어의 경계를 판단할 수 있습니다. 이는 interior boundary scoring에 해당합니다. 

Cohesion을 이야기하기 전에 한국어의 어절의 구조에 대해서 먼저 이야기를 해봅시다. 한국어의 어절은 L + [R] 구조입니다. 띄어쓰기가 제대로 되어있다면 한국어는 의미를 지니는 단어 (명사,동사,형용사,부사)가 어절의 왼쪽 (L part)에, 문법기능을 하는 (조사,어미)가 어절의 오른쪽에 등장합니다. 단, 했었습니다 = '하/동사 + 앴/선어말어미 + 습니다/어말어미'처럼 복잡하게 문법기능을 나누지 말고, '했/동사 + 었습니다/어미'로 간단히 분류합시다. '했다'를 '하다'와 다른 동사로 보면 그만입니다. '-었습니다'는 키워드 추출이나 연관어 검색 같은 데이터분석에서는 쓰지 않는 부분입니다. 분석에 필요하지 않는 부분까지 정성들일 필요는 없습니다. 그리고 새로운 단어들이 생겨나는 것은 명사,동사,형용사,부사 입니다. 이들을 open class라고 합니다. 새로운 개념이 생기기 때문에 새로운 단어가 생깁니다. 하지만 문법은 사회적 약속이고, 이것은 쉽게 바뀌지 않습니다. 이를 closed class라 합니다. 그렇다면 우리가 데이터에서 추출해야 하는 새로운 개념의 단어들은 L part입니다. 

Substrings에 대하여 카운팅을 해봅시다. [노래가 (50), 노래는 (30), 노래를 (20), 노란색 (100)]이라는 어절들이 있었다고 하면 각 어절의 왼쪽에서부터 시작하는 모든 substrings의 빈도수를 계산합니다. 

    노 (200)
    노래 (100)
    노란 (100)
    노래가 (50)
    노래는 (30)
    노래를 (20)
    노란색 (100)
    
래가, 래는, 래를, 란색 과 같이 어절의 왼쪽에서 시작하지 않은 substrings는 카운팅할 필요가 없습니다. 왜냐면 애초에 단어가 아니기 때문입니다. 

또 한가지, 우리는 길이가 1인 글자에는 그리 관심이 없습니다. 사실 한글자짜리 단어는 뽑아놓고도 해석이 힘듭니다. '이' 라는 단어는 명사/지시사/조사/숫자 등 너무 다양한 표현이 있습니다. 하지만 '이빨'처럼 두글자만 되어도 인식이 가능합니다. (심지어 한국어는 300개 정도의 글자가 문서의 거의 99%에 해당합니다. 한국어의 단어는 글자들의 조합으로 만들어집니다.) 그래서 1음절 단어는 무시합니다. 

P(노래|노)를 정의해봅시다. '노'라는 글자가 나온 다음, '노래'가 나온 경우가 100 / 200번 있었으므로 P(노래|노) = 0.5 입니다. P(노란|노)도 0.5 입니다. 하지만 P(노란색|노란) = 1입니다. cohesion(노란색)은 아래처럼 계산됩니다. 

    cohesion(노란색) = {P(노란|노) x P(노란색|노란)} ^ (1/2) = (0.5 x 1) ^ (1/2) = 0.707

1 / (n-1) power를 취하는 이유는, P(c[1:k]|c[1:k-1])는 1보다 작거나 같은 값입니다. 이를 누적해서 곱해가면 그 크기는 계속 줄어듭니다. 무조건 짧은 글자가 유리하게 됩니다. 하지만 '노란색'처럼 '노란' 다음에 반드시 '-색'이 등장한다면 이는 노란색에 점수를 더 많이 줘야하는 근거이지 않을까요? 1 / (n-1) power는 이런 효과를 줍니다. 

![cohesion](./figs/cohesion_score.JPG)


[[1]Jin, Z., & Tanaka-Ishii, K. (2006, July). Unsupervised segmentation of Chinese text by use of branching entropy. In Proceedings of the COLING/ACL on Main conference poster sessions (pp. 428-435). Association for Computational Linguistics.][branching_paper]

[[2]Feng, H., Chen, K., Deng, X., & Zheng, W. (2004). Accessor variety criteria for Chinese word extraction. Computational Linguistics, 30(1), 75-93.][accessor_paper]

[branching_paper]: https://www.researchgate.net/profile/Zhihui_Jin/publication/220873812_Unsupervised_Segmentation_of_Chinese_Text_by_Use_of_Branching_Entropy/links/561db42808aecade1acb403e.pdf
[accessor_paper]: http://www.aclweb.org/old_anthology/J/J04/J04-1004.pdf




2016년 10월 20일의 뉴스기사를 수집하여 한글만 남겨둔 전처리를 수행하였습니다. 전처리 해둔 문서는 줄바꿈을 두번 띄어쓰기로 구분해둔 데이터였기 때문에 단어열로 이뤄진 문서는 아래처럼 만들어야 합니다. 

    docs = [[sent.split() for sent in doc.split('  ')] for doc in f]
    
총 223,357개의 문장을 튜토리얼에 이용했습니다. 

In [4]:
class Sentences:
    def __init__(self, fname):
        self.fname = fname
        self.length = 0
    def __iter__(self):
        with open(self.fname, encoding='utf-8') as f:
            for doc in f:
                doc = doc.strip()
                if not doc:
                    continue
                for sent in doc.split('  '):
                    yield sent
    def __len__(self):
        if self.length == 0:
            with open(self.fname, encoding='utf-8') as f:
                for doc in f:
                    doc = doc.strip()
                    if not doc:
                        continue
                    self.length += len(doc.split('  '))
        return self.length
    
corpus_fname = 'YOUR_CORPUS'
sentences = Sentences(corpus_fname)
print('num sentences = %d' % len(sentences))

num sentences = 223357


WordExtractor는 위 세 가지 단어 가능 점수를 계산하여 줍니다. 

WordExtractor의 parameters는 다음과 같습니다. 

    def __init__(left_max_length=10, 
                right_max_length=6, 
                min_count=5, 
                verbose_points=100000, 
                min_cohesion_forward=0.1, 
                min_cohesion_backward=0.0, 
                max_droprate_cohesion=0.95, 
                max_droprate_leftside_frequency=0.95, 
                min_left_branching_entropy=0.0,
                min_right_branching_entropy=0.0, 
                min_left_accessor_variety=0, 
                min_right_accessor_variety=0, 
                remove_subwords=True)
                
train은 substrings의 빈도수를 카운팅 하는 것이며, extract는 init에 들어가는 값을 기준으로 단어를 선택하여 줍니다. 

In [44]:
%%time
from soynlp.word import WordExtractor

word_extractor = WordExtractor(min_count=100,
                               min_cohesion_forward=0.05, 
                               min_right_branching_entropy=0.0)

word_extractor.train(sentences)
words = word_extractor.extract()

training was done. used memory 0.329 Gb
all cohesion probabilities was computed. # words = 16942
all branching entropies was computed # words = 6082
all accessor variety was computed # words = 6082
CPU times: user 12.6 s, sys: 76 ms, total: 12.6 s
Wall time: 12.5 s


총 10,622개의 단어들이 추출되었습니다. 

In [49]:
len(words)

10622

words는 {word:Score} 형식의 dictionary입니다. Score는 soynlp/word.py에 구현되어있는 namedtuple입니다. 

In [58]:
print('type: %s\n' % type(words['아이오아이']))
print(words['아이오아이'])

type: <class 'soynlp.word.Scores'>

Scores(cohesion_forward=0.30063636035733476, cohesion_backward=0, left_branching_entropy=0, right_branching_entropy=0, left_accessor_variety=0, right_accessor_variety=0, leftside_frequency=270, rightside_frequency=0)


WordExtractor가 계산하는 것은 다양한 종류의 단어 가능 점수들입니다. 이를 잘 조합하여 원하는 점수를 만들 수도 있습니다. 즐겨쓰는 방법 중 하나는 cohesion_forward에 right_branching_entropy를 곱하는 것으로, (1) 주어진 글자가 유기적으로 연결되어 함께 자주 나타나고, (2) 그 단어의 우측에 다양한 조사, 어미, 혹은 다른 단어가 등장하여 단어의 우측의 branching entropy가 높다는 의미입니다. 

In [51]:
def word_score(score):
    import math
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy)\n')
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True)[:30]:
    print('%s     (%d, %.3f, %.3f)' % (word, 
                                   score.leftside_frequency, 
                                   score.cohesion_forward,
                                   score.right_branching_entropy
                                  ))

단어   (빈도수, cohesion, branching entropy)

촬영     (2222, 1.000, 1.823)
서울     (25507, 0.657, 2.241)
들어     (3906, 0.534, 2.262)
롯데     (1973, 0.999, 1.542)
한국     (9904, 0.286, 2.729)
북한     (4954, 0.766, 1.729)
투자     (4549, 0.630, 1.889)
떨어     (1453, 0.817, 1.515)
진행     (8123, 0.516, 1.970)
얘기     (1157, 0.970, 1.328)
운영     (4537, 0.592, 1.768)
프로그램     (2738, 0.719, 1.527)
클린턴     (2361, 0.751, 1.420)
뛰어     (927, 0.831, 1.298)
드라마     (2375, 0.609, 1.606)
우리     (7458, 0.470, 1.827)
준비     (1736, 0.639, 1.513)
루이     (1284, 0.743, 1.354)
트럼프     (3565, 0.712, 1.355)
생각     (3963, 0.335, 2.024)
팬들     (999, 0.626, 1.341)
산업     (2203, 0.403, 1.769)
10     (18164, 0.256, 2.210)
확인     (3575, 0.306, 2.016)
필요     (3428, 0.635, 1.279)
문제     (4737, 0.364, 1.808)
혐의     (2357, 0.962, 0.830)
평가     (2749, 0.362, 1.787)
20     (59317, 0.667, 1.171)
스포츠     (3422, 0.428, 1.604)
